In [1]:
import torch
from torch.utils.data import SubsetRandomSampler, DataLoader, RandomSampler
import torchvision
from torchvision import datasets, transforms
import os
from inclearn.lib import data, factory, losses, network, utils
import clip
import numpy as np

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda:0 device


In [7]:
model, preprocess = clip.load('RN50', device=device)
out_dim = model.text_projection.shape[1]

print("Features dimension is {}.".format(out_dim))
model

Features dimension is 1024.


CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
     

In [6]:
from inclearn.convnet.resnet import resnet50

test_model = resnet50()
test_model

Features dimension is 2048.


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# This is base code

In [8]:
model, preprocess = clip.load('RN50', device=device)
out_dim = model.text_projection.shape[1]

print("Features dimension is {}.".format(out_dim))

model.to(device)
model.eval()
torch.cuda.empty_cache()

Features dimension is 1024.


In [9]:
class_order = [87, 0, 52, 58, 44, 91, 68, 97, 51, 15, 94, 92, 10, 72, 49, 78, 61, 14, 8, 86, 84, 96, 18, 24, 32, 45, 88, 11, 4, 67, 69, 66, 77, 47, 79, 93, 29, 50, 57, 83, 17, 81, 41, 12, 37, 59, 25, 20, 80, 73, 1, 28, 6, 46, 62, 82, 53, 9, 31, 75, 38, 63, 33, 74, 27, 22, 36, 3, 16, 21, 60, 19, 70, 90, 89, 43, 5, 42, 65, 76, 40, 30, 23, 85, 2, 95, 56, 48, 71, 64, 98, 13, 99, 7, 34, 55, 54, 26, 35, 39]


cifar100_train = torchvision.datasets.CIFAR100(root="./data", train=True, transform=preprocess)
cifar100_test = torchvision.datasets.CIFAR100(root="./data", train=False, transform=preprocess)

train_list = torch.load('./exps/zero_train/train_list.pth')
test_list = torch.load('./exps/zero_train/test_list.pth')

# train_list = {}
# test_list = {}
# for i in range(100):
#     train_list[i] = []
#     test_list[i] = []
# for idx, (X,y) in enumerate(cifar100_train):
#     train_list[y].append(idx)    
# for idx, (X,y) in enumerate(cifar100_test):
#     test_list[y].append(idx)

In [10]:
feature_means = np.zeros((100,out_dim))
for class_idx in class_order:
    class_loader = DataLoader(cifar100_train, shuffle=False, batch_size=500, 
                            sampler=SubsetRandomSampler(train_list[class_idx]))
    with torch.no_grad():
        for X, y in class_loader:
            X, y = X.to(device), y.to(device)

            features = model.encode_image(X).cpu().numpy()
            feature_means[class_idx] = features.mean(axis=0)

        torch.cuda.empty_cache()

In [35]:
class_accs = []

MAX = 10000

class_nums = np.arange(50,101,1)

for i, class_idx in enumerate(class_order):
    class_loader = DataLoader(cifar100_test, shuffle=False, batch_size=100, 
                            sampler=SubsetRandomSampler(test_list[class_idx]))
    with torch.no_grad():
        for X, y in class_loader:
            X, y = X.to(device), y.numpy()

            features = model.encode_image(X).cpu().numpy()
            
            dists = -2 * (features@feature_means.T) + np.power(features, 2).sum(axis=1, keepdims=True) + np.power(feature_means, 2).sum(axis=1, keepdims=True).T
            
            class_acc = []
            for class_num in class_nums:
                if class_num < i:
                    class_acc.append(0.)
                    continue
                dists_copy = dists.copy()
                # print(i, class_idx)
                # print(class_num)
                dists_copy[:,class_order[class_num:]] = MAX
                # print(dists.shape)
                # print(dists)
                preds = dists_copy.argsort()[:,0]
                # print(preds)
                class_acc.append(np.sum(preds == y) / 100)
            
            class_accs.append(class_acc)

print(class_accs)

[[0.69, 0.69, 0.69, 0.69, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.66, 0.66, 0.66, 0.66, 0.66, 0.66, 0.66, 0.66, 0.66, 0.66, 0.66, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64], [0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76], [0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69, 0.69], [0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.

In [54]:
class_accs_np = np.array(class_accs)

S=2

inc_acc = []

for i in np.arange(50,101,S):
    # print(i)
    # print(class_accs_np[:i,i-50])
    # print(np.mean(class_accs_np[:i,i-50]))
    inc_acc.append(np.mean(class_accs_np[:i,i-50]))

print("Incremental Class Number :", S)
print("    Average Accuracy :", np.mean(inc_acc))
print("    Last Accuracy    :", inc_acc[-1])

Incremental Class Number : 2
    Average Accuracy : 0.5085646526715686
    Last Accuracy    : 0.47209999999999996


# Zero trained CLIP model accuracy function

In [1]:
def get_zero_acc(model_name):
    model, preprocess = clip.load(model_name, device=device)
    out_dim = model.text_projection.shape[1]

    print("Features dimension is {}.".format(out_dim))

    model.to(device)
    model.eval()
    torch.cuda.empty_cache()

    class_order = [87, 0, 52, 58, 44, 91, 68, 97, 51, 15, 94, 92, 10, 72, 49, 78, 61, 14, 8, 86, 84, 96, 18, 24, 32, 45, 88, 11, 4, 67, 69, 66, 77, 47, 79, 93, 29, 50, 57, 83, 17, 81, 41, 12, 37, 59, 25, 20, 80, 73, 1, 28, 6, 46, 62, 82, 53, 9, 31, 75, 38, 63, 33, 74, 27, 22, 36, 3, 16, 21, 60, 19, 70, 90, 89, 43, 5, 42, 65, 76, 40, 30, 23, 85, 2, 95, 56, 48, 71, 64, 98, 13, 99, 7, 34, 55, 54, 26, 35, 39]

    cifar100_train = torchvision.datasets.CIFAR100(root="./data", train=True, transform=preprocess)
    cifar100_test = torchvision.datasets.CIFAR100(root="./data", train=False, transform=preprocess)

    train_list = torch.load('./exps/zero_train/train_list.pth')
    test_list = torch.load('./exps/zero_train/test_list.pth')
    
    print("Obtaining feature means...")
    feature_means = np.zeros((100,out_dim))
    for class_idx in class_order:
        class_loader = DataLoader(cifar100_train, shuffle=False, batch_size=500, 
                                sampler=SubsetRandomSampler(train_list[class_idx]))
        with torch.no_grad():
            for X, y in class_loader:
                X, y = X.to(device), y.to(device)

                features = model.encode_image(X).cpu().numpy()
                feature_means[class_idx] = features.mean(axis=0)

            torch.cuda.empty_cache()
    
    print("Obtaining test accuracy...\n")
    class_acc = []

    for class_idx in class_order:
        class_loader = DataLoader(cifar100_test, shuffle=False, batch_size=100, 
                                sampler=SubsetRandomSampler(test_list[class_idx]))
        with torch.no_grad():
            for X, y in class_loader:
                X, y = X.to(device), y.numpy()

                features = model.encode_image(X).cpu().numpy()
                
                dists = -2 * (features@feature_means.T) + np.power(features, 2).sum(axis=1, keepdims=True) + np.power(feature_means, 2).sum(axis=1, keepdims=True).T
                preds = dists.argsort()[:,0]
                class_acc.append(np.sum(preds == y) / 100)

    print(class_acc)
    print()

    for inc in [10,5,2,1]:
        print("Incremental Class Number :", inc)
        avg_acc, last_acc = get_average_accuracy(class_acc,inc)
        print("    Average Accuracy :", avg_acc)
        print("    Last Accuracy    :", last_acc)

In [5]:
for model_name in clip.available_models():
    print(model_name)
    # get_zero_acc(model_name)
    # print()

RN50
RN101
RN50x4
RN50x16
RN50x64
ViT-B/32
ViT-B/16
ViT-L/14
ViT-L/14@336px


# Accuracy to Average Accuracy

CIL에서 주로 가져오는 metric이 average accuracy인데, 지금 뽑아놓은 숫자가 그냥 accuracy에 해당하기 때문에 average accuracy화 해줘야 한다.

In [10]:
import numpy as np

def get_average_accuracy(acc,inc):
    accs = []
    for i in range(50//inc+1):
        inc_acc = np.mean(acc[:50+i*inc])
        accs.append(inc_acc)

    return np.mean(accs), accs[-1]

In [16]:
acc = [0.64, 0.76, 0.69, 0.76, 0.15, 0.37, 0.7, 0.3, 0.38, 0.31, 0.65, 0.32, 0.41, 0.16, 0.77, 0.42, 0.53, 0.39, 0.56, 0.7, 0.37, 0.3, 0.46, 0.63, 0.21, 0.33, 0.23, 0.45, 0.12, 0.31, 0.53, 0.13, 0.35, 0.37, 0.39, 0.12, 0.46, 0.18, 0.6, 0.46, 0.71, 0.58, 0.55, 0.58, 0.67, 0.32, 0.15, 0.75, 0.15, 0.42, 0.55, 0.62, 0.37, 0.68, 0.47, 0.74, 0.76, 0.72, 0.39, 0.38, 0.32, 0.3, 0.55, 0.08, 0.42, 0.54, 0.69, 0.25, 0.65, 0.57, 0.76, 0.49, 0.61, 0.55, 0.62, 0.62, 0.4, 0.33, 0.18, 0.65, 0.39, 0.42, 0.69, 0.59, 0.56, 0.41, 0.55, 0.76, 0.78, 0.24, 0.63, 0.5, 0.71, 0.19, 0.31, 0.17, 0.7, 0.27, 0.47, 0.76]

for inc in [10,5,2,1]:
    print("Incremental Class Number :", inc)
    avg_acc, last_acc = get_average_accuracy(acc,inc)
    print("    Average Accuracy :", avg_acc)
    print("    Last Accuracy    :", last_acc)

Incremental Class Number : 10
    Average Accuracy : 0.4599574735449736
    Last Accuracy    : 0.47209999999999996
Incremental Class Number : 5
    Average Accuracy : 0.46005958799073776
    Last Accuracy    : 0.47209999999999996
Incremental Class Number : 2
    Average Accuracy : 0.46065413325291826
    Last Accuracy    : 0.47209999999999996
Incremental Class Number : 1
    Average Accuracy : 0.46095812526214125
    Last Accuracy    : 0.47209999999999996


In [3]:
import torch
from torchvision import models

resnet_18_pretrained = models.resnet18(pretrained=True)
resnet_18_pretrained.fc.out_features

1000

In [4]:
resnet_50_pretrained = models.resnet50(pretrained=True)
resnet_50_pretrained.fc.out_features

/home/wslee/miniconda3/envs/clvision/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1000

# 230528 `zero_train.py` 확인

In [1]:
import torch
from torch.utils.data import SubsetRandomSampler, DataLoader, RandomSampler
import torchvision
from torchvision import datasets, transforms
import os
from inclearn.lib import data, factory, losses, network, utils
import clip
import numpy as np
import sys
import tqdm


from inclearn.convnet import resnet

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda:0 device


In [2]:
model_name = 'resnet18'
n_samples = 500

In [3]:
if model_name == 'resnet18':
    model = resnet.resnet18(pretrained=True)
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    out_dim = model.out_dim
elif model_name == 'resnet50':
    model = resnet.resnet50(pretrained=True)
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    out_dim = model.out_dim
elif model_name == 'resnet18_fc':
    model = torchvision.models.resnet18(pretrained=True)
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    out_dim = model.fc.out_features
elif model_name == 'resnet50_fc':
    model = torchvision.models.resnet50(pretrained=True)
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    out_dim = model.fc.out_features
else:
    model, preprocess = clip.load(model_name, device=device)
    out_dim = model.text_projection.shape[1]

print("Features dimension is {}.".format(out_dim))

model.to(device)
model.eval()
torch.cuda.empty_cache()

Features dimension is 512.
Loading pretrained network
Features dimension is 512.


In [4]:
class_order = [87, 0, 52, 58, 44, 91, 68, 97, 51, 15, 94, 92, 10, 72, 49, 78, 61, 14, 8, 86, 84, 96, 18, 24, 32, 45, 88, 11, 4, 67, 69, 66, 77, 47, 79, 93, 29, 50, 57, 83, 17, 81, 41, 12, 37, 59, 25, 20, 80, 73, 1, 28, 6, 46, 62, 82, 53, 9, 31, 75, 38, 63, 33, 74, 27, 22, 36, 3, 16, 21, 60, 19, 70, 90, 89, 43, 5, 42, 65, 76, 40, 30, 23, 85, 2, 95, 56, 48, 71, 64, 98, 13, 99, 7, 34, 55, 54, 26, 35, 39]

cifar100_train = torchvision.datasets.CIFAR100(root="./data", train=True, transform=preprocess)
cifar100_test = torchvision.datasets.CIFAR100(root="./data", train=False, transform=preprocess)

train_list = torch.load('./exps/zero_train/train_list.pth')
test_list = torch.load('./exps/zero_train/test_list.pth')

In [5]:
print("Obtaining feature means...")
feature_means = np.zeros((100,out_dim))
for class_idx in class_order:
    class_loader = DataLoader(cifar100_train, shuffle=False, batch_size=n_samples, 
                            sampler=SubsetRandomSampler(train_list[class_idx]))
    with torch.no_grad():
        for X, y in class_loader:
            X, y = X.to(device), y.to(device)

            if model_name in ['resnet18', 'resnet50']:
                features = model(X)["features"].cpu().numpy()
            elif model_name in ['resnet18_fc', 'resnet50_fc']:
                features = model(X).cpu().numpy()
            else:
                features = model.encode_image(X).cpu().numpy()
            feature_means[class_idx] = features.mean(axis=0)

        torch.cuda.empty_cache()

Obtaining feature means...


In [6]:
print("Obtaining test accuracy...\n")
class_accs = []

MAX = 100000000000000

class_nums = np.arange(50,101,1)

for i, class_idx in enumerate(class_order):
    class_loader = DataLoader(cifar100_test, shuffle=False, batch_size=100, 
                            sampler=SubsetRandomSampler(test_list[class_idx]))
    with torch.no_grad():
        for X, y in class_loader:
            X, y = X.to(device), y.numpy()

            if model_name in ['resnet18', 'resnet50']:
                features = model(X)["features"].cpu().numpy()
            elif model_name in ['resnet18_fc', 'resnet50_fc']:
                features = model(X).cpu().numpy()
            else:
                features = model.encode_image(X).cpu().numpy()

            dists = -2 * (features@feature_means.T) + np.power(features, 2).sum(axis=1, keepdims=True) + np.power(feature_means, 2).sum(axis=1, keepdims=True).T

            class_acc = []
            for class_num in class_nums:
                if class_num < i:
                    class_acc.append(0.)
                    continue
                dists_copy = dists.copy()
                dists_copy[:,class_order[class_num:]] = MAX
                preds = dists_copy.argsort()[:,0]
                class_acc.append(np.sum(preds == y) / 100)
                # print(i,class_num)
                # print(dists)
                # if class_num == 50: print(i, np.min(dists_copy))
                # print(preds)
                # print(y)

            class_accs.append(class_acc)

class_accs_np = np.array(class_accs)

Obtaining test accuracy...



In [9]:
feature_means.shape

(100, 2048)

In [10]:
class_accs_np.shape

(100, 51)

In [11]:
final = [model_name]

for inc in [10,5,2,1]:
    inc_acc = []
    for i in np.arange(50,101,inc):
        inc_acc.append(np.mean(class_accs_np[:i,i-50]))

    print("Incremental Class Number :", inc)
    print("    Average Accuracy :", np.mean(inc_acc))
    print("    Last Accuracy    :", inc_acc[-1])
    
    if inc == 10: final.append(inc_acc[-1] * 100)
    final.append(np.mean(inc_acc) * 100)

for item in final:
    print(item, end=",")


def get_average_accuracy(acc,inc):
    accs = []
    for i in range(50//inc+1):
        inc_acc = np.mean(acc[:50+i*inc])
        accs.append(inc_acc)

    return np.mean(accs), accs[-1]

Incremental Class Number : 10
    Average Accuracy : 0.20062962962962963
    Last Accuracy    : 0.17290000000000005
Incremental Class Number : 5
    Average Accuracy : 0.19968144831370277
    Last Accuracy    : 0.17290000000000005
Incremental Class Number : 2
    Average Accuracy : 0.19959054297154963
    Last Accuracy    : 0.17290000000000005
Incremental Class Number : 1
    Average Accuracy : 0.1994735818634481
    Last Accuracy    : 0.17290000000000005
resnet18,17.290000000000006,20.062962962962963,19.968144831370278,19.959054297154964,19.94735818634481,

# 230611 Orthogonal Projection Method & zero fine-tuning

In [1]:
import torch
from torch.utils.data import SubsetRandomSampler, DataLoader, RandomSampler
import torchvision
from torchvision import datasets, transforms
import os
from inclearn.lib import data, factory, losses, network, utils
import clip
import numpy as np
from PIL import Image
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

from inclearn.lib.network import OrthogonalNet

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda:0 device


In [2]:
saved_pth = 'results/pretrained/20230611-0026_full_inc10_ep20_lr0.01_wd0.01-ft_ep20_lr0.05-k5/net_0_task_0.pth'
# saved_pth = 'results/pretrained/20230611-0026_full_inc10_ep20_lr0.01_wd0.01-ft_ep20_lr0.05-k5/net_1_task_0.pth'
# saved_pth = 'results/pretrained/20230611-0026_full_inc10_ep20_lr0.01_wd0.01-ft_ep20_lr0.05-k5/net_2_task_0.pth'

model = OrthogonalNet(
    backbone_name='ViT-B/32',
    k_orth=5,
    postprocessor_kwargs={
        'type':'learned_scaling',
        'initial_value':1.0,
    }
)
model.add_classes(50)
model.load_state_dict(torch.load(saved_pth))

model.train_classifier()
model.eval()
model.to(device)
print()

Features dimension is 512.



In [3]:
n_samples = 20
model_name = 'Orthogonal'

In [4]:
# class_order = [87, 0, 52, 58, 44, 91, 68, 97, 51, 15, 94, 92, 10, 72, 49, 78, 61, 14, 8, 86, 84, 96, 18, 24, 32, 45, 88, 11, 4, 67, 69, 66, 77, 47, 79, 93, 29, 50, 57, 83, 17, 81, 41, 12, 37, 59, 25, 20, 80, 73, 1, 28, 6, 46, 62, 82, 53, 9, 31, 75, 38, 63, 33, 74, 27, 22, 36, 3, 16, 21, 60, 19, 70, 90, 89, 43, 5, 42, 65, 76, 40, 30, 23, 85, 2, 95, 56, 48, 71, 64, 98, 13, 99, 7, 34, 55, 54, 26, 35, 39]
# class_order = [58, 30, 93, 69, 21, 77, 3, 78, 12, 71, 65, 40, 16, 49, 89, 46, 24, 66, 19, 41, 5, 29, 15, 73, 11, 70, 90, 63, 67, 25, 59, 72, 80, 94, 54, 33, 18, 96, 2, 10, 43, 9, 57, 81, 76, 50, 32, 6, 37, 7, 68, 91, 88, 95, 85, 4, 60, 36, 22, 27, 39, 42, 34, 51, 55, 28, 53, 48, 38, 17, 83, 86, 56, 35, 45, 79, 99, 84, 97, 82, 98, 26, 47, 44, 62, 13, 31, 0, 75, 14, 52, 74, 8, 20, 1, 92, 87, 23, 64, 61]
class_order = [71, 54, 45, 32, 4, 8, 48, 66, 1, 91, 28, 82, 29, 22, 80, 27, 86, 23, 37, 47, 55, 9, 14, 68, 25, 96, 36, 90, 58, 21, 57, 81, 12, 26, 16, 89, 79, 49, 31, 38, 46, 20, 92, 88, 40, 39, 98, 94, 19, 95, 72, 24, 64, 18, 60, 50, 63, 61, 83, 76, 69, 35, 0, 52, 7, 65, 42, 73, 74, 30, 41, 3, 6, 53, 13, 56, 70, 77, 34, 97, 75, 2, 17, 93, 33, 84, 99, 51, 62, 87, 5, 15, 10, 78, 67, 44, 59, 85, 43, 11]

out_dim = model.out_dim

preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
        transforms.Resize(224, interpolation=BICUBIC),
        transforms.CenterCrop(size=(224, 224)),
    ])

cifar100_train = torchvision.datasets.CIFAR100(root="./data", train=True, transform=preprocess)
cifar100_test = torchvision.datasets.CIFAR100(root="./data", train=False, transform=preprocess)

train_list = torch.load('./exps/zero_train/train_list.pth')
test_list = torch.load('./exps/zero_train/test_list.pth')

print("Obtaining feature means...")
feature_means = np.zeros((100,out_dim))
for class_idx in class_order:
    class_loader = DataLoader(cifar100_train, shuffle=False, batch_size=n_samples, 
                            sampler=SubsetRandomSampler(train_list[class_idx]))
    with torch.no_grad():
        for X, y in class_loader:
            X, y = X.to(device), y.to(device)

            features = model(X)["features"].cpu().numpy()
            feature_means[class_idx] = features.mean(axis=0)

        torch.cuda.empty_cache()

print("Obtaining test accuracy...\n")
class_accs = []

MAX = 100000000000000

class_nums = np.arange(50,101,1)

for i, class_idx in enumerate(class_order):
    class_loader = DataLoader(cifar100_test, shuffle=False, batch_size=100, 
                            sampler=SubsetRandomSampler(test_list[class_idx]))
    with torch.no_grad():
        for X, y in class_loader:
            X, y = X.to(device), y.numpy()

            features = model(X)["features"].cpu().numpy()

            dists = -2 * (features@feature_means.T) + np.power(features, 2).sum(axis=1, keepdims=True) + np.power(feature_means, 2).sum(axis=1, keepdims=True).T

            class_acc = []
            for class_num in class_nums:
                if class_num < i:
                    class_acc.append(0.)
                    continue
                dists_copy = dists.copy()
                dists_copy[:,class_order[class_num:]] = MAX
                preds = dists_copy.argsort()[:,0]
                class_acc.append(np.sum(preds == y) / 100)
                # print(i,class_num)
                # print(dists)
                # if class_num == 50: print(i, np.min(dists_copy))
                # print(preds)
                # print(y)

            class_accs.append(class_acc)

class_accs_np = np.array(class_accs)

Obtaining feature means...
Obtaining test accuracy...



In [5]:
final = [model_name]

for inc in [10,5,2,1]:
    inc_acc = []
    for i in np.arange(50,101,inc):
        inc_acc.append(np.mean(class_accs_np[:i,i-50]))

    print("Incremental Class Number :", inc)
    print("    Average Accuracy :", np.mean(inc_acc))
    print("    Last Accuracy    :", inc_acc[-1])
    
    if inc == 10: final.append(inc_acc[-1] * 100)
    final.append(np.mean(inc_acc) * 100)

for item in final:
    print(item, end=",")


def get_average_accuracy(acc,inc):
    accs = []
    for i in range(50//inc+1):
        inc_acc = np.mean(acc[:50+i*inc])
        accs.append(inc_acc)

    return np.mean(accs), accs[-1]

Incremental Class Number : 10
    Average Accuracy : 0.6216302248677249
    Last Accuracy    : 0.5727
Incremental Class Number : 5
    Average Accuracy : 0.6211320970528049
    Last Accuracy    : 0.5727
Incremental Class Number : 2
    Average Accuracy : 0.6205212592940375
    Last Accuracy    : 0.5727
Incremental Class Number : 1
    Average Accuracy : 0.6202117864952157
    Last Accuracy    : 0.5727
Orthogonal,57.269999999999996,62.16302248677249,62.11320970528049,62.05212592940374,62.021178649521566,